In [1]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

In [2]:
def getPushshiftData(after, before):
    sub = "india"
    url = 'https://api.pushshift.io/reddit/search/submission/?size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [3]:
def collectSubData(subm):
    subData = list() #list to store data points
    sub_id = subm['id']
    title = subm['title']  
    
    try:
        body = subm['selftext']
    except KeyError:
        body = "" 
    
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN" 
    
    numComms = subm['num_comments']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) # time
    score = subm['score']
    over_age = subm['over_18']
    author = subm['author']

    subData.append((title,body,flair,numComms,created,score,over_age,author))
    subStats[sub_id] = subData

In [4]:
#before and after dates
before = "1538352000" #October 1st 2018
after = "1527811200"  #June 1st 2018
subCount = 0
subStats = {}

In [5]:
data = getPushshiftData(after, before)

# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?size=1000&after=1527811200&before=1538352000&subreddit=india
1000
2018-06-03 12:08:33
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528007913&before=1538352000&subreddit=india
1000
2018-06-05 14:26:01
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528188961&before=1538352000&subreddit=india
1000
2018-06-07 14:37:22
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528362442&before=1538352000&subreddit=india
1000
2018-06-09 15:23:05
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528537985&before=1538352000&subreddit=india
1000
2018-06-11 18:55:28
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528723528&before=1538352000&subreddit=india
1000
2018-06-13 22:15:33
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1528908333&before=1538352000&subreddit=india
1000
2018-06-16 09:47:30
https://api.pushshift.io/reddit/search/su

674
2018-10-01 05:29:07
https://api.pushshift.io/reddit/search/submission/?size=1000&after=1538351947&before=1538352000&subreddit=india
0


In [6]:
subStats = list(subStats.values())

In [10]:
allowed_tags = ['AskIndia','NaN','Non-Political','Politics','Policy/Economy','Sports','Not in English.','Food','Science/Technology','Business/Finance','Demonetization','All CAPS.','Low-effort self-post.','Photography','[R]eddiquette']

extracted_dict = {
    "TITLE":[], 
    "BODY":[], 
    "FLAIR":[], 
    "NO_OF_COMMENTS":[], 
    "TIMESTAMP":[], 
    "UPVOTES":[], 
    "OVER_AGE":[], 
    "AUTHOR":[]
}

for sub in subStats:
    sub[0][0].replace(",","")
    sub[0][1].replace(",","")
    #sub[0][4].replace(",","")
    sub[0][7].replace(",","")
    extracted_dict["TITLE"].append(sub[0][0].replace(",",""))
    extracted_dict["BODY"].append(sub[0][1].replace(",",""))
    
    if sub[0][2] == '[R]eddiquette':
        extracted_dict["FLAIR"].append("Reddiquette")
    elif sub[0][2] not in allowed_tags:
        extracted_dict["FLAIR"].append("Other")
    else:
        extracted_dict["FLAIR"].append(sub[0][2])
           
    extracted_dict["NO_OF_COMMENTS"].append(sub[0][3])
    extracted_dict["TIMESTAMP"].append(sub[0][4])
    extracted_dict["UPVOTES"].append(sub[0][5])
    extracted_dict["OVER_AGE"].append(sub[0][6])
    extracted_dict["AUTHOR"].append(sub[0][7].replace(",",""))

pandas_data = pd.DataFrame(extracted_dict)
#print(pandas_data)

pandas_data.to_csv('extracted_data/data.csv', index=False)

    